In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"s3://vasveena-test-demo/jars/iceberg-spark3-runtime-0.11.0.jar",
             "spark.driver.extraClassPath":"/home/hadoop/iceberg-spark3-runtime-0.11.0.jar",
             "spark.executor.extraClassPath":"/home/hadoop/iceberg-spark3-runtime-0.11.0.jar",
             "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
             "spark.sql.catalog.spark_catalog":"org.apache.iceberg.spark.SparkSessionCatalog",
             "spark.sql.catalog.spark_catalog.type":"hive",
             "spark.sql.catalog.local":"org.apache.iceberg.spark.SparkCatalog",
             "spark.sql.catalog.local.type":"hadoop",
             "spark.sql.catalog.local.warehouse":"s3://vasveena-test-demo/iceberg/catalog/tables/"
           } 
}

In [2]:
spark.version

VBox()

Starting Spark application


The code failed because of a fatal error:
	Session 10 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
Exception in thread "main" java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.amazon.ws.emr.hadoop.fs.EmrFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2637)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3324)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3356)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:123)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3407)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3375)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:486)
	at org.apache.spark.deploy.DependencyUtils$.resolveGlobPath(DependencyUtils.scala:191)
	at org.apache.spark.deploy.DependencyUtils$.$anonfun$resolveGlobPaths$2(DependencyUtils.scala:147)
	at org.apache.spark.deploy.DependencyUt

In [3]:
val input_df = spark.read.parquet("s3://vasveena-test-demo/tmp/hudi-perf/input/")
input_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 5 more fields]
res2: Long = 2700000000


In [4]:
input_df.createOrReplaceTempView("inputdf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
import org.apache.spark.sql.functions._

val input_df2=(input_df.withColumn("z", substring(md5(concat($"id")),1,1))
                       .withColumn("schema-v", lit("v1")).withColumn("data-v", lit("v2"))
                       .withColumn("trade_dt", substring($"modified_timestamp",1,10)))
input_df2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
input_df2: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema-v|data-v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000000|    3|4000000|[E]|6e505939-f5fd-4ab...|2019|2021-04-02 00:05:02|  9|      v1|    v2|2021-04-02|
|4000001|    9|4000001|[F]|20486aca-2759-43f...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000002|   11|4000002|[G]|42962a21-a2dc-40d...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000003|    9|4000003|[H]|9841ad6d-1532-496...|2019|2021-04-02 00:05:02|  c|      v1|    v2|2021-04-02|
|4000004|    4|4000004|[I]|ff1a855a-cced-495...|2019|2021-04-02 00:05:02|  4|      v1|    v2|2021-04-02|
+-------+-----+-------+---+---

In [9]:
val input_df4 = input_df2.withColumnRenamed("schema-v", "schema_v").withColumnRenamed("data-v", "data_v")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df4: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]


In [ ]:
spark.sql("drop table local.db.iceberg_table_sparksqldf22buck")

In [8]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf22buck (id bigint,
                                       month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.object-storage.path'='s3://vasveena-test-hmswh/')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
CLUSTERED BY (uuid) INTO 10 BUCKETS
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf22buck'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res7: org.apache.spark.sql.DataFrame = []


In [9]:
val t1 = System.nanoTime
input_df4.sortWithinPartitions("z","schema_v","data_v","trade_dt").writeTo("local.db.iceberg_table_sparksqldf22buck").overwritePartitions()
//expected - no error 
//local sort instead of global sort 
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Writing job aborted.
  at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:413)
  at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:361)
  at org.apache.spark.sql.execution.datasources.v2.OverwritePartitionsDynamicExec.writeWithV2(WriteToDataSourceV2Exec.scala:306)
  at org.apache.spark.sql.execution.datasources.v2.OverwritePartitionsDynamicExec.run(WriteToDataSourceV2Exec.scala:314)
  at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:39)
  at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:39)
  at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.doExecute(V2CommandExec.scala:54)
  at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
  at org.apache.spark.sql.execution.SparkPlan.$anon

In [22]:
// This block was run before adding the JAR to driver/exec classpath and Spark lib path on EMR master node  

import org.apache.iceberg.spark.IcebergSpark
import org.apache.spark.sql.types.DataTypes

IcebergSpark.registerBucketUDF(sparkSession, "iceberg_bucket10", DataTypes.StringType, 10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:27: error: object iceberg is not a member of package org.apache
       import org.apache.iceberg.spark.IcebergSpark
                         ^



<b> Added iceberg-spark3-runtime-0.11.0.jar to /usr/lib/spark/jars,  restarted the kernel and re-ran the blocks

In [29]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf24buck (id bigint,
                                       month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.object-storage.path'='s3://vasveena-test-hmswh/')
PARTITIONED BY (bucket(10, uuid),z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf24buck'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res38: org.apache.spark.sql.DataFrame = []


In [30]:
import org.apache.iceberg.spark.IcebergSpark
import org.apache.spark.sql.types.DataTypes

IcebergSpark.registerBucketUDF(spark, "iceberg_bucket10", DataTypes.StringType, 10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.iceberg.spark.IcebergSpark
import org.apache.spark.sql.types.DataTypes


In [31]:
val t1 = System.nanoTime

input_df4.sortWithinPartitions(expr("iceberg_bucket10(uuid)"),col("z"),col("schema_v"),col("data_v"),col("trade_dt")).writeTo("local.db.iceberg_table_sparksqldf24buck").append()

val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 5615478160436
duration: String = 561.422428453seconds


In [6]:
spark.sql("select * from local.db.iceberg_table_sparksqldf24buck limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+--------+---+--------------------+----+-------------------+---+--------+------+----------+
|      id|month|      sk|txt|                uuid|year| modified_timestamp|  z|schema_v|data_v|  trade_dt|
+--------+-----+--------+---+--------------------+----+-------------------+---+--------+------+----------+
|61000029|   11|61000029|[Z]|c45ef45d-547f-4cf...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|61000202|   11|61000202|[Q]|6e7f7d83-aeb7-469...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|61000206|    8|61000206|[U]|0032c34f-29f0-49e...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|61000323|    6|61000323|[H]|9c8bc057-4921-4ac...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|61000559|    7|61000559|[J]|c1a60f24-80ca-4c0...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|61000957|   10|61000957|[R]|f2b7ec42-6c08-491...|2019|2021-04-02 00:05:02|  0|      v1|    v2|2021-04-02|
|61000966|    9|61000966|[A]|7bc7e168

In [7]:
spark.sql("select count(*) from local.db.iceberg_table_sparksqldf24buck limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|  count(1)|
+----------+
|2700000000|
+----------+

